1절에서 만들어본 인공신경망의 성능을 더 높여보자. 먼저 데이터를 가져와보자.

In [3]:
from tensorflow import keras
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


픽셀값을 0 ~ 1 사이 값으로 변화시키고 일차원 배열로 펼쳐보자.
그리고 테스트 세트와 훈련 세트로 나눠보자.

In [4]:
from sklearn.model_selection import train_test_split
train_scaled = train_input / 255.0
train_scaled = train_scaled.reshape(-1, 28 * 28)
train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled, train_target, test_size = 0.2, random_state = 42)

이번에는 인공 신경망 모델에 층을 2개 추가해보자.

1절을 만든 신경망 모델에서 입력층과 출력층 사이에 밀집층을 추가할 것이다. 이렇게 입력층와 출력층 사이에 있는 모든 층을 **'은닉층(hidden layer)'**이라고 부른다.

은닉층에는 **활성화 함수가 존재**하며 이 활성화 함수는 신경망 층의 선형 방정식의 계산 값에 적용하는 함수이다. 이전에 출력층에 적용했던 소프트맥스 함수도 활성화 함수이다. 출력층에 적용하는 활성화 함수는 이진 분류일 경우 시그모이드 함수, 다중 분류일 경우 소프트맥수 함수를 사용하는 등 종류가 제한된다. **하지만 은닉층의 활성화 함수는 비교적 자유롭다.**

그런데 은닉층에 왜 활성화 함수를 적용할까? 아래 두 선형 방정식을 보자.

* a × 4 + 2 = b
* b × 3 - 5 = c

이때 위 식을 아래의 식에 대입하면 b는 사라진다. 즉, b가 하는 일이 없는 셈이다.

* a × 12 + 1 = c

신경망도 마찬가지이다. 은닉층에서 선형적인 산술 계산만 수행하면 수행 역할이 없는 셈이다. **선형 계산을 적당하게 비선형적으로 비틀어야 한다.** 그래야 다음 층의 계산과 단순히 합쳐지지 않고 나름의 역할을 수행할 수 있다. 아래의 과정과 비슷하다.

* a × 4 + 2 = b
* log(b) = k
* k × 3 - 5 = c (b를 대입하는 것이 아니라 k를 대입함.)

많이 사용되는 활성화 함수중 하나는 '시그모이드 함수'이다. 이 함수는 뉴런의 출력 z값을 0 과 1사이로 압축한다. 그럼 시그모이드 활성화 함수를 사용한 은닉층과 소프트맥스 함수를 사용한 출력층을 케라스의 Dense 클래스로 만들어보자. 신경망의 첫 번째 층은 input_shape 매개변수로 입력의 크기를 지정해야한다.


In [5]:
dense1 = keras.layers.Dense(100, activation = 'sigmoid', input_shape = (784, ))
dense2 = keras.layers.Dense(10, activation = 'softmax')

dense1이 은닉층이고 100개의 뉴런을 가진 밀집층이다. 활성화 함수를 'sigmoid'로 지정했고 input_shape 매개변수에서 입력의 크기를 (784,)로 지정했다. 은닉층의 뉴런 개수를 정하는 데는 특별한 기준이 없다. 하지만 적어도 **출력층의 뉴런보다는 많게 만들어야한다**.

이제 두 객체를 Sequential 클래스에 추가하여 **'심층 신경망(deep nueral network, DNN)'**을 만들어 보자.

In [6]:
model = keras.Sequential([dense1, dense2])

Sequential 클래스의 객체를 만들 때 여러 개의 층을 추가하려면 리스트로 전달하면된다. 단, 출력층을 가장 마지막에 두어야 한다.

케라스 모델의 **summary() 메서드**를 호출하면 층에 대한 정보를 얻을 수 있다.

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               78500     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


맨 첫 줄에 모델의 이름이 나온다. 그 다음 이 모델의 층이 순서대로 출력된다.

층 마다 층 이름, 클래스, 모델 파라미터 개수가 출력된다. 층을 만들때 **name 매개변수**로 이름을 지정할 수 있고 지정하지 않으면 자동으로 'dense'라고 이름을 붙인다.

출력 크기를 보면 (None, 100)이다. 첫 번째 차원은 **샘플의 개수**를 나타낸다. 샘플의 개수가 아직 정의되지 않아 None이다. 케라스 모델의 fit() 메서드에 훈련 데이터를 주입하면 이 데이터를 **미니배치 경사 하강법으로 훈련**한다.

케라스의 **기본 미니배치 크기는 32개**이며 fit() 메서드의 **batch_size 매개변수**로 바꿀 수 있다. 따라서 샘플 개수를 고정하지 않고 어떤 배치 크기에도 유연하게 대응할 수 있도록 None으로 설정한다. 이렇게 신경망 층에 입력되거나 출력되는 배열의 첫 번째 차원을 **배치 차원**이라고 부른다.

두 번째 차원은 은닉층의 뉴런 개수를 100개로 두었으니 100개의 출력이 나온다. 즉 샘플마다 784개의 픽셀값이 은닉층을 통과하면서 100개의 특성으로 압축된다.

마지막으로 모델 파라미터 개수가 출력된다.

모델을 훈련하기 전에 Sequential 클래스에 층을 추가하는 다른 방법을 알아보자. 앞에서는 Dense 클래스 객체 dense1과 dense2를 만들어 리스트로 전달했다. 사실 이 두 객체를 따로 저장하여 쓸 일이 없다. 따라서 아래처럼 **Sequential 클래스의 생성자 안에서 바로 Dense 클래스의 객체를 만드는 경우가 많다**.

In [8]:
model = keras.Sequential([
                          keras.layers.Dense(100, activation = 'sigmoid', input_shape = (784,), name = 'hidden'),
                          keras.layers.Dense(10, activation = 'softmax', name = 'output')
], name = '패션 MNIST 모델')

이렇게 작업하면 추가되는 층을 한눈에 쉽게 알아볼 수 있다. summary() 메서드를 사용해보자.

In [9]:
model.summary()

Model: "패션 MNIST 모델"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden (Dense)               (None, 100)               78500     
_________________________________________________________________
output (Dense)               (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


이전과 동일한 수치들이 나왔고 name 매개변수에 전달한 이름 역시 올바르게 출력되었다. 

이 방법도 편리하지만 아주 많은 층을 추가하려면 Sequential 클래스 생성자가 너무 길어진다. Sequential 클래스에서 층을 추가할 때는 모델의 **add() 메서드**를 사용한다.

In [10]:
model = keras.Sequential()
model.add(keras.layers.Dense(100, activation = 'sigmoid', input_shape = (784,)))
model.add(keras.layers.Dense(10, activation = 'softmax'))

summary() 메서드를 사용해보자.

In [11]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 100)               78500     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


모델을 훈련해보자. compile() 메서드의 설정은 1절과 동일하다.

In [12]:
model.compile(loss = 'sparse_categorical_crossentropy', metrics = 'accuracy')
model.fit(train_scaled, train_target, epochs = 5)

Epoch 1/5
1500/1500 [==============================] - 4s 2ms/step - loss: 0.5628 - accuracy: 0.8069
Epoch 2/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.4075 - accuracy: 0.8532
Epoch 3/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3731 - accuracy: 0.8654
Epoch 4/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3496 - accuracy: 0.8727
Epoch 5/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3323 - accuracy: 0.8796


훈련세트의 성능을 보면 추가된 층이 성능을 향상시킨 것을 확인할 수 있다. 인공 신경망에 몇 개의 층을 추가해도 compile() 메서드와 fit() 메서드의 사용법은 동일하다.

이번에는 이미지 분류 문제에서 높은 성능을 낼 수 있는 활성화 함수에 대해 알아보자. 

초창기 인공 신경망의 은닉층에 많이 사용된 활성화 함수는 시그모이드 함수였다. 하지만 이 함수는 오른쪽과 왼쪽 끝으로 갈수록 그래프가 누워있기 때문에 올바른 출력을 만드는데 신속하게 대응하지 못한다.

특히 층이 많은 심층 신경망일수록 그 효과가 누적되어 학습을 더 어렵게한다. 이를 위해 **'렐루(ReLU) 함수'**가 제안되었다. 렐루 함수는 입력이 양수일 경우 활성화 함수가 없는 것처럼 입력을 통과시키고 음수일 경우 0으로 만든다.

렐루 함수는 **max(0, z)**로 쓸 수 있다. z가 0보다 크면 z를 출력하고 0보다 작으면 0을 출력한다. 렐루 함수는 특히 이미지 처리에서 좋은 성능을 낸다고 알려져있다.

렐루 함수를 사용하기전 케라스에서 제공하는 기능 하나를 살펴보자.

패션 MNIST 데이터는 28 × 28 크기 이므로 인공 신경망에 주입하기 위해 넘파이 배열의 reshape()메서드를 사용해 1차원으로 펼쳤다. 직접 이렇게 1차원으로 펼쳐도 좋지만 케라스에서는 이를 위한 **Flatten 층**을 제공한다.

Flatten 클래스는 배치 차원을 제외하고 나머지 입력 차원을 모두 일렬로 펼치는 역할만 한다. 즉, 인공 신경망의 성능을 위해 기여하는 것은 없다.

Flatten 층과 렐루 함수를 사용해보자.

In [13]:
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape = (28, 28)))
model.add(keras.layers.Dense(100, activation = 'relu'))
model.add(keras.layers.Dense(10, activation = 'softmax'))

summary() 메서드를 호출해보자.

In [14]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 100)               78500     
_________________________________________________________________
dense_5 (Dense)              (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


Flatten 클래스에 포함된 모델 파라미터는 0개이다. 케라스의 Flatten 층을 신경망 모델에 추가하면 입력값의 차원을 짐작할 수 있다. 즉, 앞의 출력에서 784개의 입력이 첫 번째 은닉층에 전달된다는 것을 알 수 있다.

훈련 데이터를 다시 준비해서 모델을 훈련해보자.

In [15]:
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()
train_scaled = train_input / 255.0
train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled, train_target, test_size = 0.2, random_state = 42)

In [16]:
model.compile(loss = 'sparse_categorical_crossentropy', metrics = 'accuracy')
model.fit(train_scaled, train_target, epochs = 5)

Epoch 1/5
1500/1500 [==============================] - 4s 2ms/step - loss: 0.5353 - accuracy: 0.8115
Epoch 2/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3932 - accuracy: 0.8598
Epoch 3/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3545 - accuracy: 0.8725
Epoch 4/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3335 - accuracy: 0.8798
Epoch 5/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3199 - accuracy: 0.8865


시그모이드 함수를 사용했을 때보다 성능이 조금 향상되었다. 검증 세트에서의 성능도 확인해보자.

In [17]:
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 1s 1ms/step - loss: 0.3642 - accuracy: 0.8758


[0.3641807436943054, 0.8758333325386047]

1절에서 은닉층을 추가하지 않은 경우보다 성능이 향상되었다.

추가적으로 인공 신경망의 하이퍼파라미터에 대해 알아보자.

하이퍼파라미터란 모델이 학습하지 않아 사람이 지정해주어야 하는 파라미터이다. 이번 절에서 어떤 하이퍼파라미터가 있었는지 생각해보자.

* 은닉층의 개수
* 은닉층의 뉴런 개수
* 활성화 함수
* 층의 종류
* batch_size 매개변수
* epochs 매개변수
* compile() 메서드의 RMSprop

compile() 메서드에서는 케라스의 기본 경사 하강법 알고리즘인 **RMSprop**을 사용했다. 케라스는 다양한 종류의 경사 하강법 알고리즘을 제공하며 이들을 **'옵티마이저(optimizer)'**라고 부른다. 위 하이퍼파라미터 외에도 RMSprop의 학습률 또한 조정할 하이퍼파라미터 중 하나이다.

즉, 처음부터 모델을 구성하고 각종 하이퍼파라미터의 최적값을 찾는 것은 어려운 작업이다. 이번에는 여러가지 옵티마이저를 테스트해보자. 가장 기본적인 옵티마이저는 확률적 경사 하강법인 **SGD**이다. 이름은 SGD이지만 기본적으로 미니배치를 사용한다.

SGD 옵티마이저를 사용하려면 compile()메서드의 **optimizer 매개변수를 'sgd'로 지정**하면 된다.

* model.compile(optimizer = 'sgd', loss = 'sparse_categorical_crossentropy', metrics = 'accuracy')

이 옵티마이저는 **tensorflow.keras.optimizers 패키지 아래 SGD 클래스**로 구현되어있다. 'sgd' 문자열은 이 클래스의 기본 설정 매개변수로 생성한 객체와 동일하다. 즉 다음 코드는 위와 동일하다.

* sgd = keras.optimizers.SGD()
* model.compile(optimizer = sgd, loss = 'sparse_categorical_crossentropy', metrics = 'accuracy')

만약 SGD 클래스의 학습률 기본값이 0.01일 때 이를 바꾸고 싶다면 **learning_rate 매개변수**에 학습률을 지정해야한다.

* sgd = keras.optimizers.SGD(learning_rate = 0.1)

SGD 외에도 다양한 옵티마이저들이 있다. 기본 경사 하강법 옵티마이저는 모두 SGD 클래스에서 제공한다. SGD 클래스의 momentum 매개변수의 기본값은 0이다. 이를 0보다 큰 값으로 지정하면 이전의 그레이디언트를 가속도처럼 사용하는 **'모멘텀 최적화(momentum optimization)'**를 사용한다. 보통 momentum 매개변수는 **0.9 이상**을 지정한다. SGD 클래스의 **nesterov 매개변수**를 기본값 False에서 True로 바꾸면 **'네스테로프 모멘텀 최적화(nesterov momentum optimization)'**를 사용한다.

* sgd = keras.optimizers.SGD(momentum = 0.9, nesterov = True)

네스테로프 모멘텀은 모멘텀 최적화를 2번 반복하여 구현한다. 일반적으로 네스테로프 모멘텀 최적화가 기본 확률적 경사 하강법보다 더 나은 성능을 제공한다.

모델이 최적점에 가까이 갈수록 학습률을 낮출 수 있다. 이렇게 하면 안정적으로 최적점에 수렴할 가능성이 높다. 이런 학습률을 **'적응적 학습률(adaptive learning rate)'**라고 한다. 이런 방식들은 학습률 매개변수를 튜닝하는 수고를 덜 수 있다.

적응적 학습률을 제공하는 대표적인 옵티마이저는 Adagrad와 RMSprop이다. 각각 compile() 메서드의 optimizer 매개변수에 **'adagard'**와 **'rmsprop'**로 지정할 수 있다. optimizer 매개변수의 기본값이 'rmsprop'이다. 이 두 옵티마이저의 매개변수를 바꾸고 싶다면 SGD와 같이 Adagrad와 RMSprop 클래스 객체를 만들어 사용하면 된다.

* adagrad = keras.optimizers.Adagrad()
* model.compile(optimizer = adagrad, loss = 'sparse_categorical_crossentropy', metrics = 'accuracy')

RMS도 동일한 방식으로 사용할 수 있다.

모멘텀 최적화와 RMSprop의 장점을 접목한 것이 Adam이다. Adam은 RMSprop와 함께 처음 시도해 볼 수 있는 좋은 알고리즘이다. 적응적 합습률을 사용하는 이 3개의 클래스의 learning_rate 매개변수 기본값은 0.001이다.

Adam 클래스의 기본값을 사용해 패션 MNIST 모델을 훈련해보자.

In [18]:
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape = (28, 28)))
model.add(keras.layers.Dense(100, activation = 'relu'))
model.add(keras.layers.Dense(10, activation = 'softmax'))

In [21]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = 'accuracy')
model.fit(train_scaled, train_target, epochs = 5)

Epoch 1/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3936 - accuracy: 0.8586
Epoch 2/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3502 - accuracy: 0.8740
Epoch 3/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3251 - accuracy: 0.8816
Epoch 4/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3035 - accuracy: 0.8891
Epoch 5/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2917 - accuracy: 0.8925


RMSprop를 사용했을 때와 거의 같은 성능을 보여준다. 검증 세트에서의 성능도 확인해보자.

In [22]:
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 1s 1ms/step - loss: 0.3312 - accuracy: 0.8823


[0.33124950528144836, 0.8823333382606506]

# 정리

핵심 포인트
* 심층 신경망: **2개 이상의 층을 포함한 신경망이다.** 종종 다층 인공 신경망, 심층 신경망, 딥러닝을 같은 의미로 사용한다.
* 렐루 함수: **이미지 분류 모델의 은닉층에서 많이 사용하는 활성화 함수이다.** 시그모이드 함수는 층이 많을수록 활성화 함수의 양쪽 끝에서 변화가 작기 때문에 학습이 어려워진다. 렐루 함수는 이런 문제가 없으며 계산도 간단하다.
* 옵티마이저: **신경망의 가중치와 절편을 학습하기 위한 알고리즘 또는 방법**을 말한다. 케라스에는 다양한 경사 하강법 알고리즘이 구현되어 있다. 대표적으로 SGD, 모멘텀, RMSprop, Adam 등이 있다.

핵심 패키지와 함수

TensorFlow
* add(): **케라스 모델에서 층을 추가하는 메서드**이다.
* summary(): **케라스 모델의 정보를 출력하는 메서드**이다.
* SGD: **기본 경사 하강법 옵티마이저 클래스**이다. **learning_rate 매개변수**로 학습률을 지정하며 기본값은 0.01이다. **momentum 매개변수**에 0 이상의 값을 지정하면 모멘텀 최적화를 수행한다. **nesterov 매개변수**를 True로 설정하면 네스테로프 모멘텀 최적화를 수행한다.
* Adagrad: **Adagrad 옵티마이저 클래스**이다. **learning_rate 매개변수**로 학습률을 지정하며 기본값은 0.001이다. Adagard는 그레이디언트 제곱을 누적하여 학습률을 나눈다. **initial_accumulator_value 매개변수**에서 누적 초깃값을 지정할 수 있으며 기본값은 0.1이다.
* RMSprop: **RMSprop 옵티마이저 클래스**이다. **learning_rate 매개변수**로 학습률을 지정하며 기본값은 0.001이다. Adagrad처럼 그레이디언트 제곱으로 학습률을 나누지만 최근의 그레이디언트를 사용하기위해 지수 감소를 사용한다. **rho 매개변수**에서 감소 비율을 지정하며 기본값은 0.9이다.
* Adam: **Adam 옵티마이저 클래스**이다. **learning_rate 매개변수**로 학습률을 지정하며 기본값은 0.001이다. 모멘텀 최적화에 있는 그레이디언트의 지수 감소 평균을 조절하기 위해 **beta_1 매개변수**가 있으며 기본값은 0.9이다. RMSprop에 있는 그레이디언트 제곱의 지수 감소 평균을 조절하기 위해 **beta_2 매개변수**가 있으며 기본값은 0.999이다.